In [93]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import json
import geopandas

In [127]:
df_cereals_2017 = pd.read_pickle("./df_cereals_2017.pkl")
df_cereals_2017

,Area,Area Code,Export Quantity,Export Value,Import Quantity,Import Value,diff Quantity,Profit
0,Afghanistan,2,0.0,0.0,764225.0,328540.0,-764225.0,-328540.0
1,Albania,3,37.0,113.0,290347.0,66264.0,-290310.0,-66151.0
2,Algeria,4,38.0,45.0,12903612.0,2750357.0,-12903574.0,-2750312.0
3,American Samoa,5,0.0,0.0,0.0,0.0,0.0,0.0
4,Angola,7,802.0,79.0,652965.0,249331.0,-652163.0,-249252.0
...,...,...,...,...,...,...,...,...
201,Venezuela (Bolivarian Republic of),236,15.0,8.0,2619310.0,801825.0,-2619295.0,-801817.0
202,Viet Nam,237,5902654.0,2661259.0,12500502.0,2555828.0,-6597848.0,105431.0
203,Yemen,249,93.0,36.0,4219671.0,1094647.0,-4219578.0,-1094611.0
204,Zambia,251,330436.0,99451.0,76798.0,33826.0,253638.0,65625.0


In [128]:
min_profit = df_cereals_2017.Profit.min()
max_profit = df_cereals_2017.Profit.max()

Json can be found ===> [here:](https://github.com/python-visualization/folium/blob/master/examples/data/world-countries.json) <====

In [129]:
countries = geopandas.read_file('data/world-countries.json')

In [130]:
cm = countries.merge(df_cereals_2017, left_on='name', right_on='Area')
cm['Profit'] = cm['Profit'].apply(lambda value : f'{int(value):,}')
cm

,id,name,geometry,Area,Area Code,Export Quantity,Export Value,Import Quantity,Import Value,diff Quantity,Profit
0,AFG,Afghanistan,"POLYGON ((61.21082 35.65007, 62.23065 35.27066...",Afghanistan,2,0.0,0.0,764225.0,328540.0,-764225.0,"-328,540"
1,AGO,Angola,"MULTIPOLYGON (((16.32653 -5.87747, 16.57318 -6...",Angola,7,802.0,79.0,652965.0,249331.0,-652163.0,"-249,252"
2,ALB,Albania,"POLYGON ((20.59025 41.85540, 20.46317 41.51509...",Albania,3,37.0,113.0,290347.0,66264.0,-290310.0,"-66,151"
3,ARE,United Arab Emirates,"POLYGON ((51.57952 24.24550, 51.75744 24.29407...",United Arab Emirates,225,224484.0,139428.0,3116863.0,1371914.0,-2892379.0,"-1,232,486"
4,ARG,Argentina,"MULTIPOLYGON (((-65.50000 -55.20000, -66.45000...",Argentina,9,40226539.0,6944919.0,20756.0,35152.0,40205783.0,"6,909,767"
...,...,...,...,...,...,...,...,...,...,...,...
141,VUT,Vanuatu,"MULTIPOLYGON (((167.84488 -16.46633, 167.51518...",Vanuatu,155,0.0,0.0,14343.0,13598.0,-14343.0,"-13,598"
142,YEM,Yemen,"POLYGON ((53.10857 16.65105, 52.38521 16.38241...",Yemen,249,93.0,36.0,4219671.0,1094647.0,-4219578.0,"-1,094,611"
143,ZAF,South Africa,"POLYGON ((31.52100 -29.25739, 31.32556 -29.401...",South Africa,202,2400015.0,576121.0,3464631.0,1004185.0,-1064616.0,"-428,064"
144,ZMB,Zambia,"POLYGON ((32.75937 -9.23060, 33.23139 -9.67672...",Zambia,251,330436.0,99451.0,76798.0,33826.0,253638.0,"65,625"


In [131]:
cm.to_file("data/countries-profit.json", driver='GeoJSON')

In [132]:
geo_json_data = json.load(open('data/countries-profit.json'))

In [133]:
def my_color_function(feature):
    """Maps high values to green and negative values to red."""
    if len(df_cereals_2017[df_cereals_2017['Area'] == feature['properties']['name']]['Profit']) == 0:
        return '#ffffff'
    value = df_cereals_2017[df_cereals_2017['Area'] == feature['properties']['name']]['Profit'].iloc[0]
    if value > 0:
        green = hex(127 + int((value / max_profit) * 255 / 2 ))[-2:]
        return '#00' + green + '00'
    else:
        red = hex(int(127 + (value / min_profit) * 255 / 2 ))[-2:]
        return '#' + red + '00' + '00'

def my_highlight_function(feature):
    return {
        'fillColor': 'black',
        'color' : 'black',
        'weight' : 3,
        'dashArray' : '5, 5'
        }
    

m = folium.Map([150,-200], zoom_start=2)

folium.GeoJson(
    geo_json_data,
    style_function=lambda feature: {
        'fillColor': my_color_function(feature),
        'color' : 'black',
        'weight' : 1.5,
        'dashArray' : '5, 5'
        },
    highlight_function=my_highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['name', 'Profit'],
        aliases=['State name:', 'Profit:'],
    )
    ).add_to(m)

m

In [134]:
m.save('data/countries_profit.html')

In [138]:
def show_production_countries(df, profit=True, filename='data/countries_profit.html'):
    look_word = 'Profit' if profit else 'diff Quantity'
    min_profit = df[look_word].min()
    max_profit = df[look_word].max()

    countries = geopandas.read_file('data/world-countries.json')
    cm = countries.merge(df, left_on='name', right_on='Area')
    cm[look_word] = cm[look_word].apply(lambda value : f'{int(value):,}')
    val_or_q = 'Value' if profit else 'Quantity'
    cm['Export ' + val_or_q] = cm['Export ' + val_or_q].apply(lambda value : f'{int(value):,}')
    cm['Import ' + val_or_q] = cm['Import ' + val_or_q].apply(lambda value : f'{int(value):,}')
    cm.to_file("data/countries-production.json", driver='GeoJSON')
    geo_json_data = json.load(open('data/countries-production.json'))

    def my_color_function(feature):
        """Maps high values to green and negative values to red."""
        if len(df[df['Area'] == feature['properties']['name']][look_word]) == 0:
            return '#ffffff'
        value = df[df['Area'] == feature['properties']['name']][look_word].iloc[0]
        if value > 0:
            green = hex(127 + int((value / max_profit) * 255 / 2 ))[-2:]
            return '#00' + green + '00'
        else:
            red = hex(int(127 + (value / min_profit) * 255 / 2 ))[-2:]
            return '#' + red + '00' + '00'

    def my_highlight_function(feature):
        return {
            'fillColor': 'black',
            'color' : 'black',
            'weight' : 3,
            'dashArray' : '5, 5'
            }


    m = folium.Map([150,-200], zoom_start=2)

    folium.GeoJson(
        geo_json_data,
        style_function=lambda feature: {
            'fillColor': my_color_function(feature),
            'color' : 'black',
            'weight' : 1.5,
            'dashArray' : '5, 5'
            },
        highlight_function=my_highlight_function,
        tooltip=folium.features.GeoJsonTooltip(
            fields=['name', look_word, 'Export ' + val_or_q, 'Import ' + val_or_q],
            aliases=['State name:', look_word + ':', 'Export ' + val_or_q + ":", 'Import ' + val_or_q + ":"],
        )
        ).add_to(m)
    m.save(filename)
    return m
        
        

In [141]:
show_production_countries(df_cereals_2017, True)